# 서울, 경기도 버스 서버에 get 요청을 보내는 requester 모듈

In [2]:
# @requester.py
import requests

def get_response(is_seoul_bus, routeId):
    if is_seoul_bus:
        params = {
            'busRouteId': routeId
        }
        response = requests.get("http://m.bus.go.kr/mBus/bus/getRouteAndPos.bms", params=params)
        return response.text
    else:
        params = {
            'serviceKey': '1234567890',
            'routeId': routeId
        }
        response = requests.get("http://openapi.gbis.go.kr/ws/rest/buslocationservice", params=params)
        return response.text

print(get_response(True, 228000174)[0:500]) # 서울 버스에 대해 Sample Response 얻기. 228000174은 5000번(용인)의 routeID
print('\n\n--------------------------------------------------------------------------------------------\n\n')
print(get_response(False, 216000044)[0:500]) # 경기도 버스에 대해 Sample Response 얻기. 216000044은 M6410번의 routeID

{"error":{"errorMessage":"성공","errorCode":"0000"},"resultList":[{"stationNo":"0","trnstnid":"101000005","busRouteId":"228000174","stationNm":"명지대앞","busType":"N","station":"228000191","direction":"서울역버스환승센터","sectSpd":"0","vehList":[],"arsId":"0","section":"0","transYn":"N","routeType":"8","gpsX":"127.1885204361","busRouteNm":"5000B용인","gpsY":"37.22422625","seq":"1","sectSpdCol":"SpeedRed","lastTm":":","fullSectDist":"0","beginTm":"07:50","existYn":"N"},{"stationNo":"78025","trnstnid":"101000005


--------------------------------------------------------------------------------------------


<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><comMsgHeader/><msgHeader><queryTime>2018-04-15 16:18:40.643</queryTime><resultCode>0</resultCode><resultMessage>정상적으로 처리되었습니다.</resultMessage></msgHeader><msgBody><busLocationList><endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기73바1614</plateNo><plateType>3</plateType><remainSeatCnt>-1</remainSeatCnt><routeId>216000044</routeId><st

---
# 버스 서버에서 온 Response 를 파싱하는 parse 모듈

### 파싱 뒤에 얻고 싶은 결과물은 "전체 정류장 번호(ex. 1~70) 중 몇 번에 어떤 버스가 지나가고 있는지"

In [7]:
# @parse.py
from requester import get_response

sample_seoulbus_string = get_response(True, 228000174)
sample_gbus_string = get_response(False, 216000044)

def parse(is_seoul_bus, string):
    List = []
    result = {}
    if is_seoul_bus: # 서울 버스인 경우의 파싱 구현
        temp = string.split('plainNo')[1:]
        for i in range(len(temp)):
            if i%2 == 1:
                List.append(temp[i].split('sectSpdCol')[0])
        for s in List:
            t = s.split(''':"''')
            result[t[1].split('''"''')[0]] = t[2].split('''"''')[0]
        return result
    else: # 경기도 버스인 경우의 파싱 구현
        temp = string.split('<plateNo>')[1:]
        for i in range(len(temp)):
            List.append(temp[i].split('</stationSeq>')[0])
        for s in List:
            result[s.split('<')[0]] = s.split('Seq>')[1]
        return result

print(parse(True ,sample_seoulbus_string))
print('\n---------------------------\n')
print(parse(False ,sample_gbus_string))

{'경기78아1125': '17', '경기78아1197': '31', '경기78아1215': '40', '경기78아1151': '51', '경기78아1319': '57', '경기78아1318': '59', '경기78아1275': '76'}

---------------------------

{'경기73바1605': '29', '경기73바1612': '13', '경기73바1173': '21', '경기73바1614': '5', '경기73바1604': '38'}


---
# 지속적으로 변화되는 버스 위치를 받아서 처리하는 compare 모듈

### 변화하는 input 을 sample 로 만들어놓고 구현

예를 들어,

 * '경기73바1173': 2   
 * '경기73바1603': 30  
 * '경기73바1605': 27     
에서<br>

* '경기73바1173'
* '경기73바1603'
* '경기73바1605'
* '경기73바1639'<br>
으로,

이렇게 변화된 input 이 들어오면 다음과 같이 해석:<br>

1173, 1603, 1605 버스는 H시 M분 S초에 각각 3, 31, 28번 정류장에 도달했고, 1639 버스가 새롭게 운행을 시작했다"

In [ ]:
# @compare.py
import datetime
import os

inputs = []
inputs.append({
    '경기73바1173': '34',
    '경기73바1603': '6',
    '경기73바1606': '30',
    '경기73바1611': '25'
})
inputs.append({
    '경기73바1173': '34',
    '경기73바1603': '7',
    '경기73바1606': '30',
    '경기73바1611': '26',
    '경기73바1611': '1'
})
inputs.append({
    '경기73바1603': '8',
    '경기73바1606': '31',
    '경기73바1611': '27',
    '경기73바1611': '2'    
})

test = {
    '경기73바1173': '34',    
    '경기73바1603': '5',
    '경기73바1606': '30',
    '경기73바1611': '25'
}

def compare(route_name ,response, current):
    now = datetime.datetime.now()
    for response_key in response: # response 에는 있는데 current 에 없는 경우 : 기점에서 운행 시작
        if not response_key in current:
            file = route_name + '/' + response_key + '.txt'
            if os.path.exists(file):
                f = open(file, 'a+')
            else:
                if not os.path.exists(route_name):
                    os.mkdir(route_name)
                f = open(file, 'w+')
            f.write(now.strftime('%Y-%m-%d %H:%M:%S') + '|'+ response[response_key] + '\n')
            f.close()
    for current_key in current: # current 에는 있는데 response 에 없는 경우 : 종점에서 운행 종료
        if not current_key in response:
            file = route_name + '/' + current_key + '.txt'
            if os.path.exists(file):
                f = open(file, 'a+')
            else:
                if not os.path.exists(route_name):
                    os.mkdir(route_name)
                f = open(file, 'w+')
            f = open(file, 'a+')
            f.write(now.strftime('%Y-%m-%d %H:%M:%S') + '|'+ current[current_key] + '\n')
            f.close()
    for current_key in current:
        if not current_key in response:
            continue
        if current[current_key] != response[current_key]:
            file = route_name + '/' + current_key + '.txt'
            if os.path.exists(file):
                f = open(file, 'a+')
            else:
                if not os.path.exists(route_name):
                    os.mkdir(route_name)
                f = open(file, 'w+')
            f = open(file, 'a+')
            f.write(now.strftime('%Y-%m-%d %H:%M:%S') + '|'+ response[current_key] + '\n')
            f.close()

for i in inputs:
    compare('1000', i, test)

---
# thread 를 이용해서 반복적으로 실행

In [ ]:
# @main.py
import datetime, threading
from requester import *
from parse import *
from compare import *

current, is_seoul_bus, route_name, routeID = {}, False, '88A', 218000018
def repeat(second = 1.0):
    global current, is_seoul_bus, route_name, routeID
    if datetime.datetime.now().hour == 3: # 새벽 3시가 되면 쓰레드를 종료
        return
    response = parse(is_seoul_bus, get_response(is_seoul_bus, routeID))
    if current == {}:
        compare(route_name, response, current)
        current = response
    else:
        compare(route_name, response, current)
        current = response
    print('Repeating...')
    threading.Timer(second, repeat, [second]).start()
    
repeat(3.0)

---
# 결과물 예시

Ex) 경기73아0000.txt

2018-04-15 19:39:52|39<br>
2018-04-15 19:41:30|40<br>
2018-04-15 19:42:10|41<br>
2018-04-15 19:44:50|42<br>
2018-04-15 19:47:10|43<br>
2018-04-15 19:51:54|EOF (운행 종료)